### Se va a llevar a cabo el desarrollo del entrenamiento de 3 tipos de algoritmos basados en árboles
1. Random Forest
2. XGBoost
3. CatBoost
### Para ello, a demás de entrenarlo se va a presentar la matriz de confusión y las métricas obtenidas

## 1. Carga y Exploración del Dataset

- **Carga del CSV:**  
  Se leyó el archivo CSV para tener una primera visualización de los datos.

- **Exploración inicial:**  
  - Se mostraron las primeras filas del dataset.
  - Se revisó la información general (tipos de datos, cantidad de valores nulos) y se obtuvieron estadísticas descriptivas.

---

## 2. Análisis de las Variables

- **Identificación de columnas no informativas:**  
  - `id` y `LoanNr_ChkDgt` son identificadores y no aportan valor predictivo.
  - `Name` contiene información variable sin relevancia directa, salvo que se extraiga información adicional (como sector o tipo de empresa).

- **Variables de fecha:**  
  - `ApprovalDate` y `DisbursementDate` podrían generar fuga de información si se usan tal cual.  
  - Se recomienda transformarlas (por ejemplo, extrayendo el año, mes, etc.) o descartarlas.

---

## 3. Selección de Variables Relevantes

- **Variables potencialmente importantes para predecir la aprobación de un crédito:**  
  - **Características de la empresa:**  
    - `State`, `ApprovalFY`  
    - `NoEmp` (número de empleados)  
    - `NewExist` (empresa nueva o existente)  
    - `CreateJob` y `RetainedJob` (impacto en el empleo)  
    - `FranchiseCode`  
    - `UrbanRural`  
    - `RevLineCr`, `LowDoc`  
    - `DisbursementGross`, `BalanceGross`
  
  - **Variables relacionadas con el banco y la ubicación:**  
    - `Bank`, `BankState`, `City`  
    - Pueden ser relevantes pero es necesario codificarlas o agruparlas correctamente.

---

## 4. Análisis de Correlaciones

- **Matriz de correlación:**  
  Se calculó la correlación entre las variables numéricas y la variable objetivo `Accept` para identificar cuáles tienen mayor influencia en la aprobación del crédito.
  
- **Visualización:**  
  Se utilizó un mapa de calor para visualizar la matriz de correlación y facilitar la identificación de relaciones relevantes.

---

## 5. Preprocesamiento de Datos

- **Eliminación o transformación de columnas:**  
  - Se descartaron columnas como `id`, `LoanNr_ChkDgt` y `Name` en un primer análisis.
  - Las columnas de fechas se pueden transformar o eliminar para evitar data leakage.

- **Codificación de variables categóricas:**  
  - Se aplicó one-hot encoding a variables como `State`, `Bank`, `City` y `BankState` para convertirlas en variables numéricas.

- **Tratamiento de valores nulos:**  
  - Se identificaron y gestionaron los valores nulos, por ejemplo, mediante imputación simple (o descartándolos según el caso).

---

## 6. División del Dataset

- **Separación en conjuntos de entrenamiento y validación:**  
  - Se dividió el dataset en un conjunto de entrenamiento y uno de validación, asegurando mantener la proporción de la variable objetivo (`Accept`) para una evaluación representativa del modelo.

---


## 1. Random Forest

In [38]:
import pandas as pd

# Cargar el dataset de entrenamiento
df = pd.read_csv('../../../data/processed/df_train.csv')

# Mostrar las primeras filas
# print(df.head())
# Información general (tipos de datos y valores nulos)
#print(df.info())

# Estadísticas descriptivas
# print(df.describe(include='all'))


Preprocesamos los datos

In [ ]:
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

#Codif variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

print(df_clean.isnull().sum())
# Ejemplo simple: imputar nulos con 0 (ajusta según cada variable)
df_clean.fillna(0, inplace=True)


ApprovalFY       0
NoEmp            0
NewExist        14
CreateJob        0
RetainedJob      0
                ..
BankState_VA     0
BankState_WA     0
BankState_WI     0
BankState_WV     0
BankState_WY     0
Length: 1730, dtype: int64


División del dataset

In [40]:
from sklearn.model_selection import train_test_split

X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


Entrenamiento Random Forest

In [41]:
from sklearn.ensemble import RandomForestClassifier

class_weights = {0: 15, 1: 1}  # Ajusta estos valores según tus datos
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weights)
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 15, 1: 1}, random_state=42)

Evaluación

In [42]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

y_pred = rf.predict(X_test)

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)

print(f"Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS")

# Reporte de Clasificación (precision, recall, f1-score, etc.)
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")


Matriz de Confusión:
[[ 260  889]
 [ 190 5512]]
Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.58      0.23      0.33      1149
           1       0.86      0.97      0.91      5702

    accuracy                           0.84      6851
   macro avg       0.72      0.60      0.62      6851
weighted avg       0.81      0.84      0.81      6851

Macro F1-Score: 0.62


## 2. XGBoost

## 3. CatBoost